In [1]:
from nsepy import get_history
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
nifty = pd.read_csv('Optionpivot.csv')
nifty['month'] = pd.to_datetime(nifty['Date']).dt.strftime('%m')
nifty['day'] = pd.to_datetime(nifty['Date']).dt.strftime('%d')
nifty.tail()

,Date,Nifty,Type,Expiry,9000,9100,9200,9300,9400,9500,...,12000,12100,12200,12300,12400,12500,12600,12700,month,day
977,2018-12-24,10681.90,PE,2018-12-27,NaN,0.25,0.80,0.90,0.95,1.00,...,1311.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,24
978,2018-12-26,10748.50,CE,2018-12-27,NaN,1630.20,1521.15,1319.35,1333.55,1246.45,...,0.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,26
979,2018-12-26,10748.50,PE,2018-12-27,NaN,0.25,0.25,0.30,0.35,0.30,...,1249.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,26
980,2018-12-27,10783.65,CE,2018-12-27,NaN,1689.20,1576.45,1465.35,1364.65,1280.05,...,0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,27
981,2018-12-27,10783.65,PE,2018-12-27,NaN,0.05,0.05,0.05,0.05,0.05,...,1213.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,27


In [3]:
Table1 = nifty.groupby('month').first()
Table1['M'] = pd.to_datetime(Table1['Date']).dt.strftime('%m')
Table1['ATM_Strike'] = round(Table1['Nifty']/100)*100

Table3 = pd.DataFrame({"Date": ["0"], "Close":0, "Expiry": ["0"], "Type": ["0"], "month":0, "day": ["0"]})
Table5 = pd.DataFrame({"Date": ["0"], "Close":0, "Expiry": ["0"], "Type": ["0"], "month":0, "day": ["0"]})
for x in range (0,9):
    e=Table1.M[x]
    d=Table1.Expiry[x]
    a= int((Table1.ATM_Strike[x]))
    f1 = str(a)
    Table2 = pd.DataFrame({"Date": nifty["Date"], "Close": nifty["Nifty"], "Expiry": nifty["Expiry"], 
                           "Type": nifty["Type"], "month": nifty["month"], "day": nifty["day"], "CE_ATM": nifty[f1]})
    Table6= Table2.where((Table2.month == e) & (Table2.Expiry == d) & (Table2.Type == 'CE'))
    Table4= Table2.where((Table2.month == e) & (Table2.Expiry == d) & (Table2.Type == 'PE'))
    Table6 = Table6.dropna()
    Table4 = Table4.dropna()
    Table6.set_index("Date", inplace = True)
    Table4.set_index("Date", inplace = True)
    Table4['PE_ATM'] = Table6['CE_ATM']
    Table4['Straddle']=(Table4['CE_ATM']+Table4['PE_ATM'])
    Table4['P&L']=Table4['Straddle'].diff().cumsum()
    Table3 = Table3.append(Table4)
    Table7 = Table4.groupby('month').last()
    Table5 = Table5.append(Table7)
    
    
Table5.drop(["Date", "Close", "Type", "month", "day"], axis = 1, inplace = True)
Table5 = Table5.dropna()
Table5['Total_P&L']=Table5['P&L'].cumsum()
print (Table5)


        Expiry  CE_ATM  PE_ATM  Straddle     P&L  Total_P&L
04  2018-04-26    0.05  307.90    307.95   17.45      17.45
05  2018-05-31    2.75   20.75     23.50 -257.55    -240.10
06  2018-06-28  106.95    0.10    107.05 -172.85    -412.95
07  2018-07-26    0.05  459.80    459.85  191.80    -221.15
08  2018-08-30    0.05  270.60    270.65   -1.60    -222.75
09  2018-09-27  621.60    0.05    621.65  336.40     113.65
10  2018-10-25  975.35    0.05    975.40  629.00     742.65
11  2018-11-29    0.05  450.40    450.45   43.10     785.75
12  2018-12-27  107.55    0.10    107.65 -282.10     503.65


In [76]:
Table8.to_csv("Strategy_computation.csv", index=True, encoding='utf8')
Table10.to_csv("Strategy_computation1.csv", index=True, encoding='utf8')